<a href="https://colab.research.google.com/github/sachinmkotian/CloudxLabs_sachin_2024/blob/main/Sachin_Assignment_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Sentiment analysis using sklearn LinearSVC (Linear Support Vector Classification)
# Project by Sachin Kotian for CloudxLabs
# Date 27-Dec-2024

#import necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


In [16]:
#Install package for pre processing text
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall
!pip install googletrans

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-tbl4uia5
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-tbl4uia5
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 0f56a6502679f6750b7f0ffed98f590d1f0246bd
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess_kgptalkie: filename=preprocess_kgptalkie-0.11-py3-none-any.whl size=8175 sha256=108c6cd695eca47afab0e646cd3f501706c0248ee5d9eb495c04c0e9ad08950a
  Stored in directory: /tmp/pip-ephem-wheel-cache-7lqyoq9p/wheels/5c/94/34/99d5ff65e88b8d9a6c5e8d8652f2311d87790a61a1b7466e21
Successfully built preprocess_kgptalkie
  Attempting uninstall: preprocess_kgptalkie
    Found existing installation: preprocess_kgptalkie 0.11
    Uninstalling preprocess_kgptalkie-0.11:
      Successfully uninstalled preprocess_kgptalkie-0.11


In [2]:
#import text preprocessors and define function for the same

import preprocess_kgptalkie as ps
import re

def get_clean(x):
  x = str(x).lower().replace('\\','').replace('_',' ')
  x = ps.contraction_to_expansion(x)
  x = ps.remove_emails(x)
  x = ps.remove_urls(x)
  x = ps.remove_html_tags(x)
  x = ps.remove_accented_chars(x)
  x = ps.remove_special_chars(x)
  x = re.sub("(.)\\1{2,}","\\1",x)
  return x


In [4]:

df = pd.read_csv("/content/amazon_review_small.txt",names=["Rating", "Summary", "Review"])
#Summary column is not required so drop it
df= df.drop(columns=['Summary'])

#Need to remove rows that don't have values
df = df.dropna(subset=["Rating","Review"])

#Run method for text preprocessing
df['Review'] = df['Review'].apply(lambda x: get_clean(x))
df.head()


/usr/local/lib/python3.10/dist-packages/preprocess_kgptalkie/text_preprocess.py:92: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(x, 'lxml').get_text()


,Rating,Review
0,1,this model may be ok for sedentary types but i...
1,4,this is a fast read filled with unexpected hum...
2,2,i bought one of these chargersthe instructions...
3,2,i was excited to find a book ostensibly about ...
4,2,i am a big jvc fan but i do not like this mode...


In [5]:
#Create categories of sentiments...either negative or positive

df["Label"] = (df.Rating >= 3).astype(int)
df = df[["Rating","Review","Label"]]

x =df['Review']
y =df['Label']

#Define Vectorization parameters
tfidf = TfidfVectorizer(max_features = 5000)
x=tfidf.fit_transform(x)

#Split dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)

#Fit the model according to the given training data
clf = LinearSVC()
clf.fit(x_train,y_train)

LinearSVC()

In [6]:
#Predict class labels for samples in x_test.
y_pred= clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.72      0.75     52259
           1       0.82      0.86      0.84     77741

    accuracy                           0.80    130000
   macro avg       0.80      0.79      0.79    130000
weighted avg       0.80      0.80      0.80    130000



In [7]:
#Summary
#This model generates sentiment on test data with accuracy of 80%